In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

In [ ]:
import pandas as pd

In [ ]:
base_url ='/input/tabular-playground-series-sep-2021'
train_url ='/kaggle/input/tabular-playground-series-sep-2021/train.csv'
test_url ='/kaggle/input/tabular-playground-series-sep-2021/test.csv'
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper

In [ ]:
mapper = DataFrameMapper([(train.columns, SimpleImputer(missing_values=np.nan, strategy='most_frequent'))])
scaled_features = mapper.fit_transform(train.copy(), 4)
scaled_features_df = pd.DataFrame(scaled_features, index=train.index, columns=train.columns)

In [ ]:
mapper = DataFrameMapper([(test.columns, SimpleImputer(missing_values=np.nan, strategy='most_frequent'))])
scaled_features_test = mapper.fit_transform(test.copy(), 4)
scaled_features_df_test = pd.DataFrame(scaled_features_test, index=test.index, columns=test.columns)

In [ ]:
type(scaled_features_df)

In [ ]:
scaled_features_df.isna().sum().sum()

In [ ]:
msk = np.random.rand(len(scaled_features_df)) < 0.8

train = scaled_features_df[msk]
vald = scaled_features_df[~msk]

In [ ]:
train_labels = train.pop('claim')
vald_labels = vald.pop('claim')
train_labels = np.array(train_labels)
vald_labels = np.array(vald_labels)

In [ ]:
column_maxes = train.max()
df_max = column_maxes.max()
normed_train_data = train / df_max

column_maxes = test.max()
df_max = column_maxes.max()
normed_valdt_data = vald / df_max

normed_train_data = np.array(normed_train_data)
normed_valdt_data = np.array(normed_valdt_data)

In [ ]:
# from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

In [ ]:
train.shape

In [ ]:
len(train.keys())/3 

In [ ]:


def get_basic_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(40, activation='relu', input_shape=[len(train.keys())]),
        tf.keras.layers.Dense(40, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
      ])

    model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=tf.keras.metrics.AUC())
    return model

In [ ]:

model = get_basic_model()

In [ ]:
model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='auc', patience=20)

In [ ]:
history = model.fit(normed_train_data, train_labels, epochs=750,
                    validation_split=0.2, batch_size=1024, verbose=1, callbacks=[early_stop])

In [ ]:
pyplot.title('Learning Curves')
pyplot.xlabel('Epoch')
pyplot.ylabel('AUC')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

In [ ]:
# Normalize test data row
column_maxes = scaled_features_df_test.max()
df_max = column_maxes.max()
normed_test_data = scaled_features_df_test / df_max

normed_test_data = np.array(normed_test_data)


In [ ]:
predict = model.predict(normed_test_data)

In [ ]:
predict

In [ ]:
sample =pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
#Making predictions and creating csv
sample['claim'] = [x[0] for x in predict.tolist()]
sample.to_csv("submission.csv", index=False)